In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import requests
from fake_useragent import UserAgent

from bs4 import BeautifulSoup
from lxml import etree
import json

import warnings
warnings.filterwarnings("ignore")

from time import sleep

import os

In [61]:
user_agent = UserAgent().firefox
main_link = 'https://otzovik.com/reviews/institut_ekonomiki_upravleniya_i_socialnih_otnosheniy_russia_moscow/'
base_link = 'https://otzovik.com'
dir_name = 'reviews'

In [5]:
res = requests.get(main_link, headers={'User-Agent': user_agent})
bs = BeautifulSoup(res.content, 'html.parser')

In [10]:
# print(bs.prettify())
links = bs.find_all('a', attrs={'class': 'review-title'})
len(links), links, [i['href'] for i in links]

(8,
 [<a class="review-title" href="/review_15522632.html" itemprop="name">Хорошие курсы повышения квалификации, меня все устроило.</a>,
  <a class="review-title" href="/review_12310367.html" itemprop="name">Не рекомендую</a>,
  <a class="review-title" href="/review_11334078.html" itemprop="name">Негативное впечатление об руководстве данного института.</a>,
  <a class="review-title" href="/review_11157254.html" itemprop="name">организация с крайне низким уровнем квалификации специалистов</a>,
  <a class="review-title" href="/review_7571570.html" itemprop="name">Не рекомендую иметь с ними дело</a>,
  <a class="review-title" href="/review_9532802.html" itemprop="name">Абсолютно непрозрачная и нечестная фирма</a>,
  <a class="review-title" href="/review_9380909.html" itemprop="name">недобросовестные поставщики услуг</a>,
  <a class="review-title" href="/review_9452114.html" itemprop="name">Отрицательное впечатление от конторы</a>],
 ['/review_15522632.html',
  '/review_12310367.html',
  '

In [26]:
link = 'https://otzovik.com/review_11334078.html'
res = requests.get(link, headers={'User-Agent': user_agent})
bs = BeautifulSoup(res.content, 'html.parser')

In [27]:
worth = bs.find_all('div', attrs={'class': 'review-plus'})
flaws = bs.find_all('div', attrs={'class': 'review-minus'})
worth[0].text, flaws[0].text

('Достоинства: никаких', 'Недостатки: непорядочность руководителя')

In [28]:
text = bs.find_all('div', attrs={'class': 'review-body description'})
text[0].text

'Работают по 100% предоплате, из - за пандемии услуги не оказали, деньги не возвращают, каждый раз кормят завтраками. Обосновать толком ничего не могут.'

In [38]:
score = bs.find_all('div', attrs={'class': 'rating-score tooltip-right'})
int(score[0].text.replace('\n', ''))

1

In [74]:
proxies = {
#  "http": "http://93.171.157.249:8080",
'http':'http://46.47.197.210:3128'
}

def get_page(url):
    res = requests.get(url, headers={'User-Agent': user_agent})#, proxies=proxies)
    bs = BeautifulSoup(res.content, 'html.parser')
    return bs

def get_links(bs):
    links = bs.find_all('a', attrs={'class': 'review-title'})
    return [i['href'] for i in links]

def get_review(bs):
    obj = {}

    worth = bs.find_all('div', attrs={'class': 'review-plus'})
    flaws = bs.find_all('div', attrs={'class': 'review-minus'})
    text = bs.find_all('div', attrs={'class': 'review-body description'})
    score = bs.find_all('div', attrs={'class': 'rating-score tooltip-right'})


    print(worth, flaws, text, score)
    obj['plus'] = worth[0].text
    obj['minus'] = flaws[0].text
    obj['text'] = text[0].text
    obj['score'] = int(score[0].text.replace('\n', ''))
    
    return obj
    

In [57]:
def get_reviews(url):
    print(url)
    bs = get_page(url)
    links = get_links(bs)
    print(links, bs)
    reviews = []
    for i in links:
        print(i)
        sleep(5)
        bs = get_page(base_link+i)
        reviews.append(get_review(bs))

    return reviews


In [75]:
# curl --proxy https://27.254.52.99:8080 icanhazip.com

x = get_reviews(main_link)
x

https://otzovik.com/reviews/institut_ekonomiki_upravleniya_i_socialnih_otnosheniy_russia_moscow/
[] <script language="JavaScript">
document.location.href = '/reviews/institut_ekonomiki_upravleniya_i_socialnih_otnosheniy_russia_moscow/?&capt4a=5591728995147810'
</script><html lang="ru">
<head>
<style type="text/css">
        *, *:before, *:after {
            -webkit-box-sizing: border-box;
            -moz-box-sizing: border-box;
            box-sizing: border-box;
        }
        html {
            height: 100%;
            -webkit-text-size-adjust: 100%;
        }
        body {
            min-height: 100%;
            height: auto !important;
            height: 100%;
            position: relative;
            font-family: "Trebuchet MS", Verdana, Arial, Helvetica, sans-serif;
            font-size: 14px;
            color: #2c2c2c;
            -webkit-tap-highlight-color: rgba(0, 0, 0, 0);
        }
    </style>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/

[]

In [62]:
os.mkdir(dir_name)

def save_reviews(file_name, reviews):
    with open(dir_name+'/'+file_name, 'a') as f:
        f.write(json.dumps(reviews))

In [63]:
save_reviews('1.txt', [{'text':12}, {'score':-1}])

In [82]:
file_names = dir_name+"/{}.txt"

def get_json(id):
    with open(file_names.format(id), 'r') as f:
        # print(f.read())
        return json.loads(f.read())

files = os.listdir(dir_name)
amount_of_files = len(files)
pages = [get_json(i) for i in range(10)]

FileNotFoundError: [Errno 2] No such file or directory: 'reviews/3.txt'

In [97]:
get_json(698)['review_list']

[{'plus': 'Достоинства: Хороший садик, отличный персонал',
  'minus': 'Недостатки: Фасад здания и отсутствие твердого специального покрытия на площадке',
  'text': 'Здравствуйте. Хочу написать отзыв о детском садике №62 Кировского района. В этот сад ходили двое моих детей, старший ходил 2 года и малышка в ясли ходила. Впечатления от садика остались самые теплые.Со старшим были вначале сложности, он у меня непоседа, адаптация проходила долго, но потом ребенок привык и всё встало на свои места. Воспитатели и помощник воспитателя были старательными, добрыми, отзывчивыми, внимательными, ребенок ходил в сад с удовольствием и я была спокойна за него.\n\nОтдельное спасибо хочу сказать воспитателям и помощнику воспитателей яслей, самые добрые, самые лучшие, самые компетентные, самые-самые хорошие воспитатели. Ребенок очень быстро прошел адаптацию, вообще не слышала чтоб дети после сентября плакали у них, моя дочка быстро привыкла к садику и всегда ходила туда с удовольствием. В садике денег с 